In [0]:
%sql
USE CATALOG nttdataeducacao;
USE SCHEMA bronze;

In [0]:
base_path = "/Volumes/nttdataeducacao/bronze/data_bronze"  

**Tabela de Resultados**

In [0]:
from pyspark.sql.functions import current_timestamp, col

df_resultados = (spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("encoding", "ISO-8859-1")
    .option("delimiter", ";")
    .csv(f"{base_path}/RESULTADOS_2024.csv")
    .select(
        "*",
        col("_metadata.file_path").alias("source_file")
        )
    .withColumn("ingestion_at", current_timestamp())
    )
display(df_resultados.limit(5)) 


In [0]:
df_resultados.write.format("delta").mode("overwrite").saveAsTable("nttdataeducacao.bronze.resultados_2024")